In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('/Users/fuhan/Desktop/Kaggle/Elo Merchant Category Recommendation/after_engineering.csv')
train_size = 201917
df_train = df[:train_size]
df_test = df[train_size:]

df_train = df_train.set_index('card_id')
df_test = df_test.set_index('card_id')

In [3]:
X = df_train.drop(['first_active_month','target'],axis=1)
y = df_train['target']

---

In [4]:
from sklearn.model_selection import KFold, cross_val_score

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X)
    rmlse= np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv = kf))
    return(rmlse)

n_folds = 5

In [5]:
import lightgbm as lgb, xgboost as xgb

model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11,verbose=-1)

In [6]:
score = rmsle_cv(model_xgb)
print('XGB')
print("Score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

XGB
Score: 3.7337 (0.0248)



In [7]:
score = rmsle_cv(model_lgb)
print('LGB')
print("Score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

LGB
Score: 3.7179 (0.0258)



In [43]:
from sklearn.linear_model import LinearRegression

model_xgb.fit(X,y)
model_lgb.fit(X,y)

X1 = model_xgb.predict(X)
X2 = model_lgb.predict(X)

X3 = pd.DataFrame({
    'xgb_pred': X1,
    'lgb_pred': X2
})

# X.drop(['level_0','index'],axis=1,inplace=True)

lr = LinearRegression()

In [9]:
kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X3)
score = np.sqrt(-cross_val_score(lr, X3, y, scoring="neg_mean_squared_error", cv = kf))
print('Essmebling')
print("Score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Essmebling
Score: 3.4436 (0.0089)



In [10]:
final_model = lr
final_model.fit(X3,y)

LinearRegression()

In [44]:
X_t = df_test.drop(['first_active_month','target'],axis=1)

X_t1 = model_xgb.predict(X_t)
X_t2 = model_lgb.predict(X_t)

In [13]:
X_t3 = pd.DataFrame({
    'xgb_pred': X_t1,
    'lgb_pred': X_t2
})

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X3_scaled = scaler.fit_transform(X3)

# X_t.set_index('card_id',inplace=True)

result = final_model.predict(X_t3)

X_t = X_t.reset_index()
out_put = X_t.copy()

out_put = out_put[['card_id']]
out_put['target'] = result

out_put.to_csv('submission.csv',index=False)

In [15]:
out_put

,card_id,target
0,C_ID_0ab67a22ab,-3.501551
1,C_ID_130fd0cbdd,-0.531972
2,C_ID_b709037bc5,-1.285393
3,C_ID_d27d835a9f,-0.216281
4,C_ID_2b5e3df5c2,-3.341864
...,...,...
123618,C_ID_7a239d2eda,1.519444
123619,C_ID_75ace375ae,-1.003447
123620,C_ID_21d56d950c,1.548685
123621,C_ID_6c46fc5a9d,-5.894048


In [21]:
X_t1

array([-2.3136117 , -0.50531197, -1.0351765 , ...,  0.7050903 ,
       -4.2805376 ,  0.01977869], dtype=float32)

In [45]:
X_t = X_t.reset_index()
out_put = X_t.copy()

out_put = out_put[['card_id']]
out_put['target'] = X_t1

In [48]:
out_put.to_csv('submission.csv',index=False)